In [3]:
import librosa
import IPython.display as player
import matplotlib.pyplot as plt
import librosa.display as disp
import os 
import numpy as np
import pandas as pd

## Feature Extraction

In [5]:
def get_mel_spectogram(filepath):
    average_len = 1293355
    aud, sr = librosa.load(filepath)
    if len(aud)<average_len:
        padding = np.zeros(average_len-len(aud))
        aud = np.concatenate((aud,padding), axis=0)
    else:
        aud = aud[:average_len]
    mel = librosa.feature.melspectrogram(y=aud, sr=sr, n_mels=128, hop_length=512, n_fft=2048)
    mel_spect = librosa.power_to_db(mel, ref=np.max)
    #mel_spect = np.expand_dims(mel_spect, axis=0)
    # target_shape = (128, 128)
    # if mel_spect.shape[1] < target_shape[1]:
    #     mel_spect = np.pad(mel_spect, ((0,0), (0,target_shape[1] - mel_spect.shape[1])), mode='constant')
    # else:
    #     mel_spect = mel_spect[:,target_shape[1]]
    #disp.specshow(mel_spect, y_axis='mel', x_axis='time')
    #return mel, aud, sr
    return mel_spect
    #return mel_spect

## Dataset Building

In [7]:
base_path = "birdclef-2023/train_audio/"

In [4]:
import pandas as pd

def build_dataset(folderpath: dict):
    #file_name = []
    class_label = []
    #music_array = []
    #sample_rate = []
    #path = []
    mel_spectogram = []
    for i in folderpath.keys():
        labellist = os.listdir(folderpath[i])
        for j in labellist:
            fp = folderpath[i]+j
            #file_name.append(j)
            class_label.append(i)
            mel_spect = get_mel_spectogram(fp)
            #music_array.append(music)
            #sample_rate.append(sr)
            mel_spectogram.append(mel_spect)
            #path.append(fp)
    #dict = {'path': path, 'file_name': file_name, 'class_label': class_label, 'music_array': music_array, 'sample_rate': sample_rate, 'mel_spectogram': mel_spectogram}
    dict = {'class_label': class_label,'mel_spectogram': mel_spectogram}
    df = pd.DataFrame(dict)
    return df

In [2]:
datasetlabelslistbinary = ['barswa', 'wlwwar']
datasetlabelslistfiveway = ['barswa', 'comsan', 'eaywag1', 'thrnig1', 'wlwwar']

In [6]:
folderpath = {}

for i in datasetlabelslistbinary:
    path = base_path+i+"/"
    #labellist = os.listdir(path)
    folderpath[i] = path

In [7]:
dataset = build_dataset(folderpath=folderpath)

In [8]:
#ataset.to_csv('DATASETS/melspecdatatwo.csv')

In [9]:
dataset

,class_label,mel_spectogram
0,barswa,"[[-43.89613687813577, -27.505238904189266, -21..."
1,barswa,"[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8..."
2,barswa,"[[-80.0, -49.18823694637195, -40.4007664036816..."
3,barswa,"[[-18.793674464748467, -2.4815546725396054, 0...."
4,barswa,"[[-65.40354146585511, -40.11949306431352, -34...."
...,...,...
995,wlwwar,"[[-77.14225236165106, -77.14225236165106, -77...."
996,wlwwar,"[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8..."
997,wlwwar,"[[-64.97996482032781, -63.975063450263825, -61..."
998,wlwwar,"[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8..."


## SVM Binary

In [12]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
label_encoder = LabelEncoder()


In [11]:
#[x.reshape(-1, 1) for x in dataset['mel_spectogram']]
X = dataset['mel_spectogram']
y = dataset['class_label']

In [12]:
X[0]

array([[-43.89613688, -27.5052389 , -21.88292966, ..., -80.        ,
        -80.        , -80.        ],
       [-36.03336787, -17.81085474, -12.80688738, ..., -80.        ,
        -80.        , -80.        ],
       [-34.2179211 , -16.69079286, -14.37048176, ..., -80.        ,
        -80.        , -80.        ],
       ...,
       [-80.        , -68.55122675, -63.07640042, ..., -80.        ,
        -80.        , -80.        ],
       [-80.        , -71.54468122, -65.59046346, ..., -80.        ,
        -80.        , -80.        ],
       [-80.        , -79.39164519, -74.44213069, ..., -80.        ,
        -80.        , -80.        ]])

In [13]:
X_flat = [x.flatten() for x in X]

In [14]:
X_flat_2d = np.array(X_flat)

In [15]:
y = label_encoder.fit_transform(y)

In [16]:
np.unique(y)

array([0, 1])

In [17]:
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_flat_2d, y, test_size=0.2, random_state=42)
# Optional: Normalize the feature vectors
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
#X_test = scaler.transform(X_test)

In [20]:
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)

SVC(C=1, kernel='linear')

In [21]:
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.72


In [22]:
def get_features(filepath):
    average_len = 1293355
    aud, sr = librosa.load(filepath)
    if len(aud)<average_len:
        padding = np.zeros(average_len-len(aud))
        aud = np.concatenate((aud,padding), axis=0)
    else:
        aud = aud[:average_len]
    chrom = librosa.feature.chroma_stft(y=aud, sr=sr)
    spec_cont = librosa.feature.spectral_centroid(y=aud, sr=sr)
    return np.concatenate((chrom.mean(axis=1), spec_cont.mean(axis=1)))
    

## SVM Five Way

In [8]:
folderpath = {}

for i in datasetlabelslistfiveway:
    path = base_path+i+"/"
    #labellist = os.listdir(path)
    folderpath[i] = path

datasetmulticlass = build_dataset(folderpath=folderpath)

In [9]:
datasetmulticlass

,class_label,mel_spectogram
0,barswa,"[[-43.89613687813577, -27.505238904189266, -21..."
1,barswa,"[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8..."
2,barswa,"[[-80.0, -49.18823694637195, -40.4007664036816..."
3,barswa,"[[-18.793674464748467, -2.4815546725396054, 0...."
4,barswa,"[[-65.40354146585511, -40.11949306431352, -34...."
...,...,...
2495,wlwwar,"[[-77.14225236165106, -77.14225236165106, -77...."
2496,wlwwar,"[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8..."
2497,wlwwar,"[[-64.97996482032781, -63.975063450263825, -61..."
2498,wlwwar,"[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8..."


In [10]:
X_ = datasetmulticlass['mel_spectogram']
y_ = datasetmulticlass['class_label']

In [13]:
X__flat = [x.flatten() for x in X_]
X__flat_2d = np.array(X__flat)
y_ = label_encoder.fit_transform(y_)

In [14]:
len(X__flat)

2500

In [15]:
len(X_)

2500

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X__flat_2d, y_, test_size=0.2, random_state=42)
# Optional: Normalize the feature vectors
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)

SVC(C=1, kernel='linear')

In [22]:
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.494
